In [1]:
import pandas as pd
import numpy as np
import os
import sys
import cv2
import random
import matplotlib.pyplot as plt
from keras_unet.models import satellite_unet
from keras_unet.models import vanilla_unet
from keras_unet.models import custom_unet
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, dice_coef
from keras_unet.losses import jaccard_distance
from keras.callbacks import ModelCheckpoint
from keras_unet.utils import plot_imgs

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import math
from keras.callbacks import CSVLogger

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [7]:
#check if gpu enabled, also make sure you have gpu version of tf. 
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Create test data by moving a subset (10%) to seperate folder
In this implementation we do a 72-18-10 split

In [4]:
import shutil
img_dir = '/path/to/images'
mask_dir = '/path/to/masks'
test_mask = '/new/path/to/test/masks'
test_img = '/new/path/to/test/images'

img_files = os.listdir(img_dir)
mask_files = os.listdir(mask_dir)
test_size = len(img_files)//10

img_files.sort()
mask_files.sort()

c = list(zip(img_files, mask_files))
random.shuffle(c)

img_files, mask_files = zip(*c)

for i in range(test_size):
    shutil.move(os.path.join(img_dir,img_files[i]),os.path.join(test_img,img_files[i]))
    shutil.move(os.path.join(mask_dir,mask_files[i]),os.path.join(test_mask,mask_files[i]))

## Data generator with augmentation
Ajust batch size according to your RAM resources. Note the folder structure below in order for the flow_from_directory to work.

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   rotation_range=5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   brightness_range=[0.6,1])
val_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

"""
---Folder structure----

-train:
        -PS-RGB #images
        -train_mask_bin #masks
-test:
        -PS-RGB #test-images
        -train_mask_bin #test-masks
"""

data_path = '/path/to/train/'
test_path = '/path/to/test/'

seed1 = 1
seed2 = 2
training_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=4,
        class_mode=None,
        classes = ['PS-RGB'],
        shuffle=True,
        subset = 'training',
        seed = seed1)

train_mask_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=4,
        class_mode=None,
        classes = ['train_mask_bin'],
        shuffle=True,
        subset= 'training',
        seed = seed1)

validation_generator = val_datagen.flow_from_directory(
        data_path,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=4,
        class_mode=None,
        classes = ['PS-RGB'],
        shuffle=True,
        subset = 'validation',
        seed = seed2)

val_mask_generator = val_datagen.flow_from_directory(
        data_path,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=4,
        class_mode=None,
        classes = ['train_mask_bin'],
        shuffle=True,
        subset = 'validation',
        seed = seed2)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=4,
        class_mode=None,
        classes = ['PS-RGB'],
        shuffle=False)

test_mask_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=4,
        class_mode=None,
        classes = ['train_mask_bin'],
        shuffle=False)

train_generator = zip(training_generator, train_mask_generator)
val_generator = zip(validation_generator, val_mask_generator)
test_gen = zip(test_generator, test_mask_generator)

Found 661 images belonging to 1 classes.
Found 661 images belonging to 1 classes.
Found 165 images belonging to 1 classes.
Found 165 images belonging to 1 classes.
Found 92 images belonging to 1 classes.
Found 92 images belonging to 1 classes.


In [6]:
#Make sure this is zero, so validation and train not have been mixed
print(len(set(test_mask_generator.filenames) & set(val_mask_generator.filenames)))

0


## Some custom metrics

In [6]:
from keras_unet import TF
if TF:
    from tensorflow.keras import backend as K
else:    
    from keras import backend as K

def threshold_binarize(x, threshold=0.3):
    ge = tf.greater_equal(x, tf.constant(threshold))
    y = tf.where(ge, x=tf.ones_like(x), y=tf.zeros_like(x))
    return y

def iou_thresholded(y_true, y_pred, threshold=0.3, smooth=1.):
    y_pred = threshold_binarize(y_pred, threshold)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)


## U-net with ResNet-blocks

In [2]:
from keras.backend import int_shape
from keras.models import Model
from keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Add, BatchNormalization, Input, Activation, Lambda, Concatenate

#res_unet function directly taken from https://github.com/Nishanksingla/UNet-with-ResBlock/blob/master/resnet34_unet_model.py

def res_unet(filter_root, depth, n_class=1, input_size=(256, 256, 1), activation='relu', batch_norm=True, final_activation='sigmoid'):
    """
    Build Reduced UNet model with ResBlock
    Args:
        filter_root (int): Number of filters to start with in first convolution.
        depth (int): How deep to go in UNet i.e. how many down and up sampling you want to do in the model. 
                    Filter root and image size should be multiple of 2^depth.
        n_class (int, optional): How many classes in the output layer. Defaults to 2.
        input_size (tuple, optional): Input image size. Defaults to (256, 256, 1).
        activation (str, optional): activation to use in each convolution. Defaults to 'relu'.
        batch_norm (bool, optional): To use Batch normaliztion or not. Defaults to True.
        final_activation (str, optional): activation for output layer. Defaults to 'softmax'.
    Returns:
        obj: keras model object
    """
    inputs = Input(input_size)
    x = inputs
    # Dictionary for long connections
    long_connection_store = {}

    if len(input_size) == 3:
        Conv = Conv2D
        MaxPooling = MaxPooling2D
        UpSampling = UpSampling2D
    elif len(input_size) == 4:
        Conv = Conv3D
        MaxPooling = MaxPooling3D
        UpSampling = UpSampling3D

    # Down sampling
    for i in range(depth):
        out_channel = 2**i * filter_root
        #out_channel = filter_root
        # Residual/Skip connection
        res = Conv(out_channel, kernel_size=1, padding='same', use_bias=False, name="Identity{}_1".format(i))(x)

        # First Conv Block with Conv, BN and activation
        conv1 = Conv(out_channel, kernel_size=3, padding='same', name="Conv{}_1".format(i))(x)
        if batch_norm:
            conv1 = BatchNormalization(name="BN{}_1".format(i))(conv1)
        act1 = Activation(activation, name="Act{}_1".format(i))(conv1)

        # Second Conv block with Conv and BN only
        conv2 = Conv(out_channel, kernel_size=3, padding='same', name="Conv{}_2".format(i))(act1)
        if batch_norm:
            conv2 = BatchNormalization(name="BN{}_2".format(i))(conv2)

        resconnection = Add(name="Add{}_1".format(i))([res, conv2])

        act2 = Activation(activation, name="Act{}_2".format(i))(resconnection)

        # Max pooling
        if i < depth - 1:
            long_connection_store[str(i)] = act2
            x = MaxPooling(padding='same', name="MaxPooling{}_1".format(i))(act2)
        else:
            x = act2

    # Upsampling
    for i in range(depth - 2, -1, -1):
        out_channel = 2**(i) * filter_root
        #out_channel = filter_root

        # long connection from down sampling path.
        long_connection = long_connection_store[str(i)]

        up1 = UpSampling(name="UpSampling{}_1".format(i))(x)
        up_conv1 = Conv(out_channel, 2, activation='relu', padding='same', name="upConv{}_1".format(i))(up1)

        #  Concatenate.
        up_conc = Concatenate(axis=-1, name="upConcatenate{}_1".format(i))([up_conv1, long_connection])

        #  Convolutions
        up_conv2 = Conv(out_channel, 3, padding='same', name="upConv{}_11".format(i))(up_conc)
        if batch_norm:
            up_conv2 = BatchNormalization(name="upBN{}_1".format(i))(up_conv2)
        up_act1 = Activation(activation, name="upAct{}_1".format(i))(up_conv2)

        up_conv2 = Conv(out_channel, 3, padding='same', name="upConv{}_2".format(i))(up_act1)
        if batch_norm:
            up_conv2 = BatchNormalization(name="upBN{}_2".format(i))(up_conv2)

        # Residual/Skip connection
        res = Conv(out_channel, kernel_size=1, padding='same', use_bias=False, name="upIdentity{}_1".format(i))(up_conc)

        resconnection = Add(name="upAdd{}_1".format(i))([res, up_conv2])

        x = Activation(activation, name="upAct{}_2".format(i))(resconnection)

    # Final convolution
    output = Conv(n_class, 1, padding='same', activation=final_activation, name='output')(x)

    return Model(inputs, outputs=output, name='Res-UNet')

In [7]:
model = res_unet(filter_root=64, depth=4, input_size= (512,512,3), n_class=1, final_activation='sigmoid')
model.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy', iou_thresholded, dice_coef])
print('Model compiled.')
model.summary()

Model compiled.
Model: "Res-UNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
Conv0_1 (Conv2D)                (None, 512, 512, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
BN0_1 (BatchNormalization)      (None, 512, 512, 64) 256         Conv0_1[0][0]                    
__________________________________________________________________________________________________
Act0_1 (Activation)             (None, 512, 512, 64) 0           BN0_1[0][0]                      
___________________________________________________________________________

## Training of model
Adjust validation and traning steps in line to your data size. Since we use the augmentation, the steps_per_epoch can be set to be rather large -> $batchsize \cdot steps$ > size of training data. For validation steps we simply set -> $batchsize \cdot steps$ = size of validation data. Parameters in the step decay can also be modified according to your preferences. 

In [6]:
def step_decay(epoch):
	initial_lrate = 1e-3
	drop = 0.2
	epochs_drop = 8
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate

model_checkpoint = ModelCheckpoint('vegas-resunet-weights-v2.h5', verbose=1, monitor='val_loss', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=10)
lrate = LearningRateScheduler(step_decay)
csv_logger = CSVLogger('vegas-resunet-log-v2.csv')

batch_size = 4
history = model.fit(
    train_generator,
    steps_per_epoch = 500,
    validation_data = val_generator, 
    validation_steps = 165//batch_size,
    epochs = 30,
    callbacks=[model_checkpoint, es, lrate, csv_logger],
    verbose=1)
print('Model trained and weights saved.')

Epoch 1/30
400/400 [==============================] - 159s 384ms/step - loss: 0.3956 - accuracy: 0.8164 - iou_thresholded: 0.1928 - dice_coef: 0.1980 - val_loss: 0.1385 - val_accuracy: 0.9471 - val_iou_thresholded: 0.4575 - val_dice_coef: 0.5169

Epoch 00001: val_loss improved from inf to 0.13851, saving model to vegas-unet-weights-v3.h5
Epoch 2/30
400/400 [==============================] - 158s 394ms/step - loss: 0.1254 - accuracy: 0.9085 - iou_thresholded: 0.3682 - dice_coef: 0.3610 - val_loss: 0.1192 - val_accuracy: 0.9494 - val_iou_thresholded: 0.4787 - val_dice_coef: 0.5355

Epoch 00002: val_loss improved from 0.13851 to 0.11919, saving model to vegas-unet-weights-v3.h5
Epoch 3/30
400/400 [==============================] - 155s 389ms/step - loss: 0.1208 - accuracy: 0.9077 - iou_thresholded: 0.3789 - dice_coef: 0.3807 - val_loss: 0.1142 - val_accuracy: 0.9531 - val_iou_thresholded: 0.5053 - val_dice_coef: 0.5306

Epoch 00003: val_loss improved from 0.11919 to 0.11424, saving model 

## Evaluating model on test data
First we make predictions based on the test images. Then we evaluate the model metrics. Finally, we save the predicted masks, ground truth masks and input imagery in a new folder. 

In [10]:
model.load_weights('vegas-unet-weights-v3.h5')
y_pred = model.predict(test_generator)

In [11]:
model.evaluate(test_gen, steps=23)

23/23 [==============================] - 3s 115ms/step - loss: 0.1166 - accuracy: 0.9541 - iou_thresholded: 0.5240 - dice_coef: 0.5238


[0.10942984372377396,
 0.9548848867416382,
 0.5492416024208069,
 0.5450347661972046]

In [12]:
pred_dir = '/new/path/to/predictions'
mask_dir = '/path/to/test/masks'
for i, image in enumerate(y_pred):
    image = (image * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(pred_dir, str(i) + '_pred.png'), image)
    cv2.imwrite(os.path.join(pred_dir, str(i) + '_ground-truth.png'), cv2.imread(os.path.join(mask_dir,test_mask_generator.filenames[i]),1))
    cv2.imwrite(os.path.join(pred_dir, str(i) + '_original.png'), cv2.imread(os.path.join(mask_dir,test_generator.filenames[i]),1))


## Model performance
Using plotly to vizualize the training process in terms of loss and accuracy

In [2]:
import plotly.express as px
hist = pd.read_csv('vegas-resunet-log-v2.csv')
fig = px.line(hist, x='epoch', y=['loss', 'val_loss'])
fig.update_layout(
    title="Training loss",
    yaxis_title="value",
    font=dict(
    size=16)
)
fig.show()

In [4]:
hist = pd.read_csv('vegas-resunet-log-v2.csv')
fig = px.line(hist, x='epoch', y=['iou_thresholded', 'val_iou_thresholded', 'dice_coef', 'val_dice_coef'])
fig.update_layout(
    title="Training accuracy",
    yaxis_title="value",
    font=dict(
    size=16)
)
fig.show()